<a href="https://colab.research.google.com/github/sayarghoshroy/Summarization/blob/master/summarization_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pickle
import spacy
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor as mlp
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install rouge
from rouge import Rouge

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# change to path to dataset
file_name = "/content/drive/My Drive/Summarization_Pickled_Data/cnn_dataset_1000_labelled.pkl"
stories = pickle.load(open(file_name, 'rb'))

In [5]:
# displaying the first datapoint
# verify correctness of load
print(stories[0])

{'story': ["potomac, maryland (cnn) -- to combat the depression and despair during her 105-day stint in iran's notorious evin prison, haleh esfandiari welcomed all distractions and blocked thoughts of her beloved home and family.", 'haleh esfandiari talks to iranian media in front of evin prison after her august 21 release.', 'the iranian-american scholar, who was charged with espionage and endangering iranian national security during a december visit to her family, wrote a book in her mind, read newspapers, watched television and exercised voraciously.', '"i decided either i am going to succumb to despair or i am going to try to make the best of this condition, and the best of this condition was to have a disciplined day," she said. "so i would exercise for many hours, i would read, i would walk a lot, some three to four hours a day -- even in the room, you know, i would pace up and down timing myself."', 'the 67-year-old grandmother of two said dwelling on her incarceration, and long

In [6]:
# Required Models for glove
# in case of errors with conda, use this:
# conda install -c conda-forge spacy
# this is what worked for me :P

!python -m spacy download en
!python -m spacy download en_core_web_lg
!python -m spacy link en_core_web_lg en --force

# use the large model as the default model for English textual data

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=2a1e8fe1183d0944b5a3009a87e62498fe756835f9527038754e95c315d4a3b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-ydki3qwh/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
# Initializing the processor
embedder = spacy.load('en')

In [0]:
# basic embeddings using averaged glove vectors
# using Spacy's large language model
def get_embedding(text):
    extract = embedder(text)
    total_sum = np.zeros(300)
    count = 0
    for token in extract:
        count += 1
        total_sum += np.asarray(token.vector)
    return total_sum / count

In [0]:
# creating the inputs and expected outputs
train_size = 900
val_size = 50
test_size = 50

def make_set(start_index, size):
    count = 0
    X_set = []
    y_set = []

    while count < size:
        data = stories[start_index + count]
        count += 1

        doc_emb = get_embedding(data['story_text'])
        # use the function of choice to generate the document embedding

        index = 0
        for sentence in data['story']:
            sent_emb = get_embedding(sentence)
            # use the function of choice to generate the sentence embedding

            x = np.concatenate((sent_emb, doc_emb))
            try:
                y = data['scores'][index]
            except:
                y = 0.0
            index += 1

            X_set.append(x)
            y_set.append(y)

    return np.asmatrix(X_set), np.asarray(y_set)

X_train, y_train = make_set(0, train_size)
X_val, y_val = make_set(train_size, val_size)
X_test, y_test = make_set(train_size + val_size, test_size)

In [0]:
def get_values(X, model):
    return model.predict(X)

def get_loss(pred, y):
    return np.linalg.norm(pred - y) / np.shape(y)[0]

model_name = "glove_averaged"
# modify the model name

def train(X_train, y_train):
    model = mlp(hidden_layer_sizes = (1024, 2048, 1024, 512, 512, 256, 128), max_iter = 1000)
    
    train_size = np.shape(X_train)[0]

    batch_size = int(np.sqrt(train_size))
    n_batches = int(4 * (train_size / batch_size))

    print("Total Number of Training Examples: " + str(train_size))
    print("Batch Size: " + str(batch_size))
    print("Number of Batches: " + str(n_batches))

    min_loss = 1e20

    while(n_batches > 0):
        idx = np.random.randint(0, train_size, size = batch_size)

        X_select = X_train[idx,:]
        y_select = y_train[idx]

        model.partial_fit(X_select, y_select)

        sentence_predicted_scores = get_values(X_val, model)

        loss = get_loss(sentence_predicted_scores, y_val)

        # saving best model seen so far
        if loss < min_loss:
            min_loss = loss
            pickle.dump(model, open(model_name + '_best_model', 'wb'))

        n_batches -= 1

    final_model = pickle.load(open(model_name + '_best_model', 'rb'))
    return final_model

In [11]:
m = train(X_train, 1000 * y_train)

Total Number of Training Examples: 19223
Batch Size: 138
Number of Batches: 557


In [0]:
# Hyperparameter for similarity threshold
theta = 0.95

def similarity(A, B):
    similarity =  (A @ B.T) / (np.linalg.norm(A) * np.linalg.norm(B))
    return similarity

def get_top_4(X_doc, y):
    order = np.flip(np.argsort(y))
    sentence_set = []
    for sent_id in order:
        if sentence_set == []:
            sentence_set.append(order[0])
            continue

        consider = X_doc[sent_id, :]
        flag = 1
        for consider_id in sentence_set:
            if similarity(X_doc[consider_id, :], consider) > theta:
                flag = 0
                break

        if flag == 1:
            sentence_set.append(sent_id)
    return sentence_set[0: min(4, len(sentence_set))]

In [0]:
# Creating object of the ROUGE class
rouge = Rouge()

In [14]:
# evaluation
# testing out each document iteratively
# test set: document 'train_size + val_size' onwards

def join(lst):
    string = ""
    for elem in lst:
        string = string + elem + " . "
    return string

def extract_rouge(rouge_dict):
    scores = []

    scores.append(100 * rouge_dict["rouge-1"]['f'])
    scores.append(100 * rouge_dict["rouge-1"]['p'])
    scores.append(100 * rouge_dict["rouge-1"]['r'])

    scores.append(100 * rouge_dict["rouge-2"]['f'])
    scores.append(100 * rouge_dict["rouge-2"]['p'])
    scores.append(100 * rouge_dict["rouge-2"]['r'])

    scores.append(100 * rouge_dict["rouge-l"]['f'])
    scores.append(100 * rouge_dict["rouge-l"]['p'])
    scores.append(100 * rouge_dict["rouge-l"]['r'])

    return np.asarray(scores)

start_doc_id = train_size + val_size
doc_count = len(stories)

generated_summary, gold_summary = 0, 0

# set the number of documents for testing
limit = test_size

total = np.zeros(9)
# averaging the 9 ROUGE Metrics

count = 0

while count < min(doc_count, limit):
    X_doc = []
    y_doc = []
    data = stories[start_doc_id + count]
    doc_emb = get_embedding(data['story_text'])

    index = 0
    for sentence in data['story']:
        sent_emb = get_embedding(sentence)

        x = np.concatenate((sent_emb, doc_emb))
        try:
            y = data['scores'][index]
        except:
            y = 0.0

        index += 1

        X_doc.append(x)
        y_doc.append(y)

    X_doc = np.asmatrix(X_doc)
    y_doc = np.asarray(y_doc)

    sentence_predicted_scores = get_values(X_doc, m)

    loss = np.linalg.norm(sentence_predicted_scores - y_doc)

    # Uncomment to view the test_loss on the sample  
    # print(loss)

    summary_sent_id = get_top_4(X_doc, sentence_predicted_scores)
    # Uncomment to view the indices of chosen sentences
    # print("Document ID:", start_doc_id + count, ", Top 5 Sentences:", summary_sent_id)

    # Uncomment to view the top 10 sentences based on Gold Labels
    # print("Top 10 sentences based on Gold Label", np.ndarray.tolist(np.flip(np.argsort(y_doc))[0:10]))

    gold_summary = join(data['highlights'])
    generated_summary = join([data['story'][idx] for idx in summary_sent_id])

    scores = rouge.get_scores(generated_summary, gold_summary)[0]
    total += extract_rouge(scores)

    count += 1

averaged = total / test_size

predicted = get_values(X_test, m)
test_loss = get_loss(y_test, predicted)

print("Sample Output:")
print("Document:\n", stories[-1]['story_text'])
print("Generated Summary:\n", generated_summary)
print("Gold Summary:\n", gold_summary)

print("\nAll Metrics:\n")

lst = np.ndarray.tolist(averaged)
lst.append(test_loss)

df = pd.DataFrame([lst], columns = ['R1-f', 'R1-p', 'R1-r',
                                    'R2-f', 'R2-p', 'R2-r',
                                    'Rl-f', 'Rl-p', 'Rl-r',
                                    'Test Regression Loss'], dtype = float)
df.index = ['Averaged Glove Vectors']
display(df)

# save results into a dataframe file
df.to_csv(model_name + '_results.csv')

Sample Output:
Document:
 -- in the wake of the earthquake in haiti, george clooney and other celebrities have signed on for a telethon to aid the devastated island nation..the oscar-winning actor will take part in a fundraising program to air commercial-free across several networks, mtv announced.."hope for haiti now" will air on abc, cbs, nbc, fox, cnn, bet, the cw, hbo, mtv, vh1 and cmt starting at 8 p.m. et/pt and 7 p.m. ct on friday, january 22..mtv said clooney will serve as a host in los angeles, while musician wyclef jean will be in new york, and cnn's anderson cooper will appear from haiti..the two-hour event will feature as-yet-unnamed musical performances and celebrity appearances, as well as live news reports from cnn..it's hollywood's latest philanthropic gesture in reaction to the catastrophic situation in haiti..a celebrity lounge at this weekend's golden globe awards in beverly hills has been turned into a haitian aid fundraiser..medecins sans frontieres says actors ang

,R1-f,R1-p,R1-r,R2-f,R2-p,R2-r,Rl-f,Rl-p,Rl-r,Test Regression Loss
Averaged Glove Vectors,20.251263,17.12711,31.217825,4.084392,3.463325,6.40485,20.449854,17.96798,28.451117,0.017769


In [0]:
# ^_^ Thank You